<a class="reference external" href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/Training/Computational-Workflows-on-DesignSafe/Jupyter_Notebooks/Jupyter_Notebooks_TapisApps/tapis_getApps_List.ipynb" target="_blank">
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# List Tapis Apps
***get list of apps from Tapis using getApps()***

by Silvia Mazzoni, DesignSafe, 2025

In this module, we’ll use the Tapis API to obtain a list of all Tapis applications available. You will need the app name and version to submit a job.

We will first get a list of all apps, then we will define some search criteria to find all apps that meet our needs, such as 'opensees' apps.

At the end we will use a python function.

## Using *getApps* with Query Parameters

The *getApps()* command in the Tapis API allows you to retrieve a list of registered applications, but it’s more than a simple list function. It supports **query parameters** that let you filter and control what gets returned — making it easier to find only the apps you care about.

For example, you can filter by:

* **App ID or partial ID**
* **Owner** (e.g., apps you've registered)
* **Execution system**
* **Category**
* **Search terms in metadata**

This is useful when working in environments like DesignSafe, where hundreds of applications may be registered.


## getApps() Query Parameters


### Tapis Query Parameters

| Name              | In    | Type         | Default             | Description                                                                                                                        |
| ----------------- | ----- | ------------ | ------------------- | ---------------------------------------------------------------------------------------------------------------------------------- |
| *search*          | query | string       | *(none)*            | Search conditions as a single query parameter. Example: *search=(id.like.MyApp*)~(enabled.eq.true)*                                |
| *listType*        | query | ListTypeEnum | *(none)*            | Filters results based on ownership, permissions, and sharing. Default is to return only items owned by the requester.              |
| *select*          | query | string       | summaryAttributes | Attributes to return in each result. Supports keywords *allAttributes* and *summaryAttributes*. Example: *select=id,version,owner* |
| *limit*           | query | integer      | 100               | Limits the number of items returned. Example: *limit=10*. Use *-1* for unlimited.                                                  |
| *orderBy*         | query | string       | *(none)*            | Attribute for sorting. May include direction, e.g., *orderBy=id(desc)*. Default is ascending order.                                |
| *skip*            | query | integer      | 0                 | Number of items to skip. Use either *skip* or *startAfter*. Example: *skip=10*.                                                    |
| *startAfter*      | query | string       | *(none)*            | Starting point when sorting. Requires *orderBy*. Example: *limit=10&orderBy=id(asc)&startAfter=my.app1*                            |
| *computeTotal*    | query | boolean      | false             | If *true*, computes the total number of matching results even when *limit* is applied.                                             |
| *showDeleted*     | query | boolean      | false             | Indicates whether to include items marked as deleted. Default is *false*.                                                          |
| *impersonationId* | query | string       | *(none)*            | Restricted. Only certain Tapis services or tenant admins may impersonate a user.                                                   |

#### Options for listType:
- *OWNED* Include only items owned by requester (Default)
- *SHARED_PUBLIC* Include only items shared publicly
- *SHARED_DIRECT* Include only items shared directly with requester. Exclude publicly shared items.
- *READ_PERM* Include only items for which requester was granter READ or MODIFY permission.
- *MINE* Include items owned or shared directly with requester. Exclude publicly shared items.
- *ALL* Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
  
<a href="https://github.com/tapis-project/tapipy/blob/main/tapipy/resources/openapi_v3-apps.yml" target="_blank">source</a>

## How to Pass a *search* Parameter to *getApps*

The query parameters use Tapis's **FIQL (Feed Item Query Language)** format — a structured way of building powerful filters into your API requests.
    
FIQL allows you to build **structured filters** in string format, using **comparison operators**, **logical connectors**, and **field names**.

The Tapis *search* parameter is **very picky**. 


Tapis expects the *search* query to be a **string** that contains a list of **valid search expressions**, each wrapped in **parentheses**, and joined by *~* (logical AND). Each condition must match the format:

```
(attribute.operator.value)
```

For example, to find apps owned by *"silvia"*:

```python
search = "(owner.eq.silvia)"
apps = client.apps.getApps(search=search)
```

### Example with Multiple Conditions

```python
search = "(owner.eq.silvia)~(id.like.opensees*)"
apps = client.apps.getApps(search=search)
```

---

### ❌ Common Mistakes

| Mistake                   | Example Mistake            | Why it's wrong                                                  |
| ------------------------- | -------------------------- | --------------------------------------------------------------- |
| Using *=* instead of *.*  | *owner=eq.silvia*          | Must be *owner.eq.silvia*                                       |
| Missing outer parentheses | *owner.eq.silvia*          | Needs to be *(owner.eq.silvia)*                                 |
| Using *owner.eq(silvia)*  | *owner.eq(silvia)*         | Wrong syntax — no parentheses around value                      |
| Not using string quotes   | *search=(owner.eq.silvia)* | Must be passed as a Python string: *search="(owner.eq.silvia)"* |

---

### Summary

To fix your code:

```python
apps = client.apps.getApps(search="(owner.eq.silvia)")
```



## Tapis Search Operators

Tapis supports a set of well-defined **search operators** for filtering resources like apps, systems, jobs, etc. 
Here's a list of the most common and useful ones, along with descriptions and examples:


| Operator   | Meaning                          | Example                             | Matches...                       |
| ---------- | -------------------------------- | ----------------------------------- | -------------------------------- |
| *eq*       | Equal to                         | *(owner.eq.silvia)*                 | *owner == "silvia"*              |
| *ne*       | Not equal to                     | *(owner.ne.silvia)*                 | *owner != "silvia"*              |
| *gt*       | Greater than                     | *(created.gt.2024-01-01T00:00:00Z)* | Dates after Jan 1, 2024          |
| *lt*       | Less than                        | *(created.lt.2024-01-01T00:00:00Z)* | Dates before Jan 1, 2024         |
| *ge*       | Greater than or equal            | *(memory.ge.32)*                    | Memory ≥ 32 GB                   |
| *le*       | Less than or equal               | *(memory.le.128)*                   | Memory ≤ 128 GB                  |
| *like*     | Pattern match (wildcards: ***)   | *(id.like.opensees*)*               | ID starts with *opensees*        |
| *in*       | In list                          | *(owner.in.silvia,joe,bob)*         | Owner is silvia OR joe OR bob    |
| *nin*      | Not in list                      | *(owner.nin.admin,test)*            | Owner is NOT admin or test       |
| *between*  | Within range (for dates/numbers) | *(memory.between.32,128)*           | Memory between 32 and 128 GB     |
| *nbetween* | Not within range                 | *(memory.nbetween.32,128)*          | Memory NOT between 32 and 128 GB |
| *isnull*   | Is null                          | *(description.isnull.true)*         | Description is null              |
| *notnull*  | Is not null                      | *(description.notnull.true)*        | Description is NOT null          |


###  Combining Conditions

Use *~* (tilde) to combine conditions (AND):

***python
search = "(owner.eq.silvia)~(id.like.opensees*)"
***

Use *|* (pipe) for OR conditions:

***python
search = "(id.eq.testapp)|(id.eq.hello-world)"
***

Note: Complex logic with multiple *AND*/*OR* groups may require careful parentheses nesting.

NOTE: DesignSafe Apps are owned by 'wma_prtl' or others.

In [1]:
import time
from tapipy.tapis import TapisResult

In [2]:
import os,sys
PathOpsUtils = os.path.expanduser('~/CommunityData/Training/Computational-Workflows-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

## Connect to Tapis

Yes, you need to first connect to Tapis, this authenticates you

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2026-02-05T18:54:18+00:00
 Token expires in: 1:52:21.351533
-- AUTHENTICATED VIA SAVED TOKEN --


## look for all apps

In [4]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         listType=listType,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2025-05-15T17:27:31.749283Z
description: Compress a file or folder for download.
id: compress
owner: wma_prtl
version: 0.0.4
--

created: 2025-05-15T17:27:32.216947Z
description: Compress a file or folder for download.
id: compress-ls6
owner: wma_prtl
version: 0.0.4
--

created: 2024-02-26T21:19:27.391841Z
description: Extract a tar, tar.gz, tgz, gz, or zip file.
id: extract
owner: wma_prtl
version: 0.0.1
--

created: 2024-02-26T21:19:27.532619Z
description: Extract a tar, tar.gz, tgz, gz, or zip file.
id: extract-ls6
owner: wma_prtl
version: 0.0.1
--

created: 2024-02-26T21:19:27.731838Z
description: Run an interactive Fiji session on Lonestar6.
id: fiji
owner: wma_prtl
version: 2.14.0
--

created: 2024-02-26T21:19:27.929974Z
description: Run an interactive Jupyter Notebook session with ability to launch mpi jobs.
id: jupyter-hpc-mpi
owner: wma_prtl
version: 1.0.1
--

created: 2024-02-26T21:19:28.097343Z
description: Run an interactive Jupyter Notebook session with abilit

## Look for all apps with opensees in their id, and the latest version

In [5]:
searchQuery = "(id.like.*opensees*)~(version.eq.latest)"
listType = 'ALL'
select = 'id,created,description,version'
orderBy = 'created(asc)'
results = t.apps.getApps(search=searchQuery,
                         orderBy=orderBy,
                         listType=listType,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2025-02-20T18:01:49.338155Z
description: Parallel version driven by a single processor. Easy to use even with limited knowledge about parallel computing.
id: opensees-sp-s3
version: latest
--

created: 2025-02-20T18:01:49.005183Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3
version: latest
--

created: 2025-02-20T18:41:03.661272Z
description: OpenSees-EXPRESS provides users with a sequential OpenSees interpreter. It is ideal to run small sequential scripts on DesignSafe resources freeing up your own machine.
id: opensees-express
version: latest
--

created: 2025-02-20T18:54:03.185268Z
description: OpenSees Interactive provides users with a sequential OpenSees interpreter. It is ideal to run small sequential scripts on DesignSafe resources freeing up your own machine.
id: opensees-interactive
version: latest
--

created: 2025-02-20T18:41:03.168289Z
descripti

## Look for more Specific app

In [6]:
searchQuery = "(id.like.*opensees*mp*)~(version.eq.latest)"
results = t.apps.getApps(search=searchQuery,
                listType=listType,select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2025-02-20T18:01:49.005183Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3
version: latest
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
version: latest


## Look for a different version

In [7]:
searchQuery = "(id.like.*opensees*mp*)~(version.eq.3.5.0)"
results = t.apps.getApps(search=searchQuery,
                listType=listType,select=select)  
for thisRes in results:
    print('--')
    print(thisRes)
    

--

created: 2025-02-26T20:29:01.705255Z
description: OpenSeesMP is an OpenSees interpreter intended for high performance computers for performing finite element simulations with parameteric studies and very large models on parallel machines. OpenSeesMP requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-ls6
version: 3.5.0


### Use a utility function:

In [8]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils,['query_tapis_apps.py'])

Output()

In [9]:
results = OpsUtils.query_tapis_apps(t,['opensees','mp'],version='latest',select = 'id,created,description,version')    
print(results)
print('------------')
results = OpsUtils.query_tapis_apps(t,['opensees','mp'],select = 'id,created,description,version')    
print(results)
print('------------')
results = OpsUtils.query_tapis_apps(t,['opensees','mp'],version='latest')    
print(results[0].id)

[
created: 2025-02-20T18:01:49.005183Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3
version: latest, 
created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
version: latest, 
created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
version: latest]
------------
[
created: 2024-02-26T21:19:29.638483Z
description: OpenSeesMP is an OpenSees interpreter intended for high performance computers for performing finite element simulations with parameteric studies and very large models on parallel machines. OpenSeesMP requires understanding of parallel processing and the capabili

In [10]:
results = OpsUtils.query_tapis_apps(t,['openseespy'],version='latest')    
print(results[0].id)

IndexError: list index out of range